In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import requests

In [2]:
def log_progress(message):
    time_format = '%Y-%m-%d %H:%M:%S'
    now = datetime.now()
    timestamps =  now.strftime(time_format)
    with open('log.txt' , 'a') as file:
        file.write(timestamps+'  '+message+'\n')

In [49]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
page = requests.get(url).text
soup = BeautifulSoup(page,'html.parser')



In [47]:
dictionary = {
    'Rank': [],
    'Bank_Name':[],
    'MC_USD_Billion':[]
}
table = soup.find('tbody')
for index , row in enumerate(table.find_all('tr')):
    
    if row and index != 0:
        cols = row.find_all('td')
        dictionary['Rank'].append(cols[0].contents[0].replace('\n' , ''))
        dictionary['Bank_Name'].append(cols[1].find_all('a')[1].contents[0].replace('\n' , ''))
        dictionary['MC_USD_Billion'].append(cols[2].contents[0].replace('\n' , ''))
        
dictionary

{'Rank': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'Bank_Name': ['JPMorgan Chase',
  'Bank of America',
  'Industrial and Commercial Bank of China',
  'Agricultural Bank of China',
  'HDFC Bank',
  'Wells Fargo',
  'HSBC Holdings PLC',
  'Morgan Stanley',
  'China Construction Bank',
  'Bank of China'],
 'MC_USD_Billion': ['432.92',
  '231.52',
  '194.56',
  '160.68',
  '157.91',
  '155.87',
  '148.90',
  '140.83',
  '139.82',
  '136.81']}

In [41]:
list(dictionary.keys())

['Rank', 'Bank_Name', 'MC_USD_Billion']

In [50]:
data = pd.DataFrame(dictionary , columns = list(dictionary.keys()))
data

,Rank,Bank_Name,MC_USD_Billion
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


In [51]:
exchange_rate_data = pd.read_csv('exchange_rate.csv')
exchange_rate_data

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


In [56]:
data['MC_USD_Billion'] = data['MC_USD_Billion'].astype(float)

In [58]:
exchange_rate_data.dtypes

Currency     object
Rate        float64
dtype: object

In [60]:
import numpy as np

In [78]:
EUR_rate = exchange_rate_data['Rate'].loc[exchange_rate_data['Currency'] == 'EUR'].values[0]
GBP_rate = exchange_rate_data['Rate'].loc[exchange_rate_data['Currency'] == 'GBP'].values[0]
INR_rate = exchange_rate_data['Rate'].loc[exchange_rate_data['Currency'] == 'INR'].values[0]
print(EUR_rate)
print(GBP_rate)
print(INR_rate)

0.93
0.8
82.95


In [79]:
import sqlite3
conn = sqlite3.connect('Banks.db')
df = pd.read_sql('Select * From Largest_Banks' , conn)
conn.close()
df

,Bank_Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,403.0,346.0,35911.0
1,Bank of America,231.52,215.0,185.0,19205.0
2,Industrial and Commercial Bank of China,194.56,181.0,156.0,16139.0
3,Agricultural Bank of China,160.68,149.0,129.0,13328.0
4,HDFC Bank,157.91,147.0,126.0,13099.0
5,Wells Fargo,155.87,145.0,125.0,12929.0
6,HSBC Holdings PLC,148.90,138.0,119.0,12351.0
7,Morgan Stanley,140.83,131.0,113.0,11682.0
8,China Construction Bank,139.82,130.0,112.0,11598.0
9,Bank of China,136.81,127.0,109.0,11348.0


In [84]:
query = "Select * From Largest_Banks"
conn = sqlite3.connect('Banks.db')
df = pd.read_sql(query, conn)
df

,Bank_Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,Bank of America,231.52,215.31,185.22,19204.58
2,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,HDFC Bank,157.91,146.86,126.33,13098.63
5,Wells Fargo,155.87,144.96,124.70,12929.42
6,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
7,Morgan Stanley,140.83,130.97,112.66,11681.85
8,China Construction Bank,139.82,130.03,111.86,11598.07
9,Bank of China,136.81,127.23,109.45,11348.39
